# Sleep-EDF PyTorch Training - Google Colab

Este notebook treina um modelo PyTorch otimizado para classificação de estágios de sono usando o dataset Sleep-EDF.

## Características:
- ✅ Modelo LSTM bidirecional com atenção
- ✅ Data loading eficiente para datasets grandes
- ✅ Progress bar com ETA durante treinamento
- ✅ Resultados reproduzíveis com seeds fixos
- ✅ Otimizações para memória e velocidade

## Requisitos:
- Google Colab com GPU habilitada
- Dados no Google Drive: `mydrive/mhealth-data/data/processed/sleep-edf/`


## 1. Configuração Inicial


In [ ]:
# Instalar dependências necessárias
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install numpy pandas scikit-learn matplotlib seaborn
!pip install pyedflib mne

# Clonar o repositório
!git clone https://github.com/vasco-fernandes21/mhealth-data-privacy.git
import sys
sys.path.append('/content/mhealth-data-privacy')

print("✅ Dependências instaladas e repositório clonado")


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 149.4 MB/s eta 0:00:00


In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verificar se os dados existem
import os
data_path = '/content/drive/MyDrive/mhealth-data/data/processed/sleep-edf'

if os.path.exists(data_path):
    print(f"✅ Dados encontrados em: {data_path}")

    # Listar arquivos
    files = os.listdir(data_path)
    print(f"📁 Arquivos disponíveis: {files}")

    # Verificar tamanhos
    for file in ['X_train.npy', 'y_train.npy', 'X_val.npy', 'y_val.npy', 'X_test.npy', 'y_test.npy']:
        if file in files:
            size = os.path.getsize(os.path.join(data_path, file))
            print(f"  {file}: {size / (1024*1024):.1f} MB")
        else:
            print(f"  ❌ {file} não encontrado")
else:
    print(f"❌ Dados não encontrados em: {data_path}")
    print("💡 Certifique-se de que os dados estão no caminho correto no Google Drive")


In [ ]:
# Executar o treinamento usando o script do repositório (preserva __file__)
import os, shutil

repo_data_dir = '/content/mhealth-data-privacy/data/processed/sleep-edf'
drive_data_dir = '/content/drive/MyDrive/mhealth-data/data/processed/sleep-edf'
models_dir = '/content/mhealth-data-privacy/models/sleep-edf/baseline_torch'
results_dir = '/content/mhealth-data-privacy/results/sleep-edf/baseline'

# Garantir diretórios de saída (evita erro no torch.save)
os.makedirs(models_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# Criar ligação simbólica dos dados do Drive para o path esperado pelo script
os.makedirs('/content/mhealth-data-privacy/data/processed', exist_ok=True)
if os.path.islink(repo_data_dir) or os.path.exists(repo_data_dir):
    try:
        if os.path.islink(repo_data_dir):
            os.unlink(repo_data_dir)
        else:
            shutil.rmtree(repo_data_dir)
    except Exception as e:
        print(f"Aviso ao remover destino antigo: {e}")

# Criar symlink
!ln -sf "$drive_data_dir" "$repo_data_dir"
print(f"✅ Dados referenciados via symlink: {repo_data_dir} -> {drive_data_dir}")

print(f"🚀 Iniciando treinamento com dados de: {drive_data_dir}")
print("=" * 80)

# Executar o script (agora __file__ está definido corretamente)
!python /content/mhealth-data-privacy/src/train/sleep-edf/train_baseline_torch.py

print("✅ Treinamento concluído!")

## 3. Análise dos Resultados


In [ ]:
# Carregar e analisar os resultados
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Carregar resultados
results_path = '/content/mhealth-data-privacy/models/sleep-edf/baseline_torch/results_sleep_edf.json'

if os.path.exists(results_path):
    with open(results_path, 'r') as f:
        results = json.load(f)

    print("📊 RESULTADOS FINAIS:")
    print("=" * 50)
    print(f"Accuracy:  {results['accuracy']:.4f}")
    print(f"Precision: {results['precision']:.4f}")
    print(f"Recall:    {results['recall']:.4f}")
    print(f"F1-Score:  {results['f1_score']:.4f}")

    # Matriz de confusão
    print("\n📋 MATRIZ DE CONFUSÃO:")
    cm = np.array(results['confusion_matrix'])
    class_names = results['class_names']

    print(f"{'':8s}", end="")
    for name in class_names:
        print(f"{name:8s}", end="")
    print("\n{'Real ↓':8s}", end="")

    for i, row in enumerate(cm):
        print(f"{class_names[i]:8s}", end="")
        for val in row:
            print(f"{val:8d}", end="")
        print()

    # Plot da matriz de confusão
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Matriz de Confusão - Sleep-EDF PyTorch')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.show()

else:
    print(f"❌ Resultados não encontrados em: {results_path}")
    print("💡 Execute primeiro a célula de treinamento")


## 💡 Dicas e Troubleshooting

### Problemas Comuns:

1. **Dados não encontrados:**
   - Verifique se o caminho `mydrive/mhealth-data/data/processed/sleep-edf/` está correto
   - Certifique-se de que todos os arquivos `.npy` estão presentes

2. **GPU não disponível:**
   - Vá em Runtime → Change runtime type → Hardware accelerator → GPU
   - O modelo funcionará na CPU, mas será mais lento

3. **Memória insuficiente:**
   - O dataset Sleep-EDF é grande (58MB para treino)
   - Considere usar batch_size menor se necessário

4. **Tempo de treinamento longo:**
   - O ETA mostrado durante o treinamento é preciso
   - Para datasets grandes, considere interromper e retomar treinamento

### Recursos Utilizados:
- **GPU**: Recomendada para treinamento rápido
- **RAM**: ~4GB para dataset + modelo
- **Disco**: ~200MB para código + resultados

### Próximos Passos:
1. Experimentar diferentes hiperparâmetros
2. Implementar técnicas de privacidade (DP-SGD)
3. Testar com outros datasets fisiológicos

---

**Notebook criado para o projeto Mestrado SIDM - MHealth Data Privacy** 🚀
